In [1]:
!pip3 install --upgrade numpy

In [2]:
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/kennong/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Price Volume data retrieval using Yahoo Finance API

Target tickers:
- S&P 500 Index (SPY)
- VIX Index (^VIX)
- Gold (GC=F)
- Brent Crude (BZ=F)
- WTI (CL=F)

In [3]:
# !apt-get install -y build-essential
# !apt-get install -y wget
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib/
# !./configure --prefix=/usr
# !make
# !make install
!pip uninstall -y numpy
!pip install numpy == 1.20.3
!pip install TA-Lib

import yfinance as yf
import talib

Found existing installation: numpy 1.26.1
Uninstalling numpy-1.26.1:
  Successfully uninstalled numpy-1.26.1
zsh:1: = not found
  Using cached numpy-1.26.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.1-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.1 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.1 which is incompatible.


In [4]:
# Create price extractor

class price_volume_data:
    """A class for retrieving and analyzing price and volume data."""
    def __init__(self, tickers, start_date, end_date):
        """
        Initializes a new instance of the price_volume_data class.

        Args:
            tickers (list): A list of tickers for the assets to retrieve data for.
            start_date (str): The start date for the data retrieval (format: 'YYYY-MM-DD').
            end_date (str): The end date for the data retrieval (format: 'YYYY-MM-DD').
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.pv = self.__retrieve_data__()
        
        #self.lp, self.ret = self.__get_daily_returns__()
        
    def __retrieve_data__(self):
        """
        Retrieves price and volume data for the given tickers from Yahoo Finance API.

        Returns:
            dict: A dictionary containing the price and volume data for each ticker.
        """

        data, failed = {}, []
        #shares_outstanding = []
        for ticker in self.tickers:
            try:
                df = yf.Ticker(ticker).history(start = self.start_date, end = self.end_date)
                #shares_outstanding.append(yf.Ticker(ticker).info['sharesOutstanding'])   
                
                data[ticker] = df
                
            except:
                failed.append(ticker)
                
        if failed:
            print('Tickers failed to extract: ', failed)
        
        return data
    
    def __get_daily_returns__(self):
        """
        Computes daily returns for the retrieved price data.

        Returns:
            pd.DataFrame: A DataFrame containing the daily price data.
            pd.DataFrame: A DataFrame containing the daily price returns.
        """
        df = pd.DataFrame()
        
        for ticker in self.pv:
            if df.empty:
                df = pd.DataFrame(self.pv[ticker]['Close'].copy())
            else:
                df = df.merge(self.pv[ticker]['Close'], how='outer',left_index=True, right_index=True)

        df.columns = self.tickers
        df = df.fillna(method = 'ffill')
        
        return df, df/df.shift(1) - 1
    
    def get_info(self, ticker = None):
        """
        Prints information about the price and volume data for the specified ticker.
        If no ticker is provided, prints information for all tickers.

        Args:
            ticker (str, optional): The ticker symbol for the asset. Defaults to None.
        """
        if ticker:
            self.pv[ticker].info()
        
        else:
            for ticker in self.pv:
                self.pv[ticker].info()
                
        return None
    
    def plot_price_trends(self, normalize = True, reference = -1):
        """
        Plots the close prices of selected assets over time.

        Args:
            normalize (bool, optional): Indicates whether to normalize the prices. Defaults to True.
            reference (int, optional): Reference point for normalization. Defaults to -1.
        """
        f, ax = plt.subplots(1,1,figsize=(12,5))
        # plot prices
        for key in self.pv:
            last_price = self.pv[key].loc[:, 'Close'].copy()
            if normalize:
                if reference == -1:
                    last_price /= last_price.iloc[0]
                elif reference == 0:
                    midpt = int(last_price.shape[0]/2)
                    last_price /= last_price.iloc[midpt]
                else:
                    last_price /= last_price.iloc[-1]
            last_price.plot(ax = ax, label = key)
        
        ax.title.set_text('Close Prices of selected assets')
        ax.set(ylabel = 'Normalized Price to start of observation')
        ax.legend()
        plt.show()
        
    def plot_corr(self, method='pearson', output = False, series = 'price'):
        """
        Plots the correlation matrix between asset prices or returns.

        Args:
            method (str, optional): The correlation method to use. Defaults to 'pearson'.
            output (bool, optional): Indicates whether to return the correlation matrix. Defaults to False.
            series (str, optional): Indicates whether to use 'price' or 'returns'. Defaults to 'price'.

        Returns:
            pd.DataFrame or None: The correlation matrix if output=True, None otherwise.
        """
        corr_matrix = self.lp.corr(method) if series == 'price' else self.ret.corr(method)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        f, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(corr_matrix, mask=mask, vmin=-1, vmax=1, annot=True)
        
        return corr_matrix if output else None
        
        
    def get_technical_indicators(self, indicators = None, normalize = True):
        """
        Computes and adds various technical indicators to the price data.

        Args:
            indicators (list, optional): A list of tuples specifying the indicators and their parameters.
                Defaults to None.
            normalize (bool, optional): Indicates whether to normalize the indicator values. Defaults to True.
        """
        if not indicators:
            indicators = [('SMA', 5), ('SMA', 14), ('ATR', 5), ('ATR', 14), ('ADX', 5),
                         ('ADX', 14), ('RSI', 5), ('RSI', 14), ('MACD', None), ('BBANDS', None)]
        
        for data in self.pv:
            df = self.pv[data]
            for indicator, param in indicators:
                
                
                if indicator == 'SMA':
                    df[f'{indicator}_{param}'] = talib.SMA(df['Close'], param)

                elif indicator == 'ATR':
                    df[f'{indicator}_{param}'] = talib.ATR(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'ADX':
                    df[f'{indicator}_{param}'] = talib.ADX(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'RSI':
                    df[f'{indicator}_{param}'] = talib.RSI(df['Close'], param) 
                    
                elif indicator == 'MACD':
                    df[f'{indicator}'], df[f'{indicator}_signal'], _ = talib.MACD(df['Close']) 
                    
                    
                elif indicator == 'BBANDS':
                    df[f'{indicator}_upp'], df[f'{indicator}_mid'], df[f'{indicator}_low'] = talib.BBANDS(df['Close'])
                    
                    
                else:
                    print('Indicators not built into the class. Please run TALIB manually!')
            
            df = df.replace([np.inf, -np.inf], np.nan, inplace = True)
            
        return None              

In [5]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM
from keras.regularizers import L1, L1L2, L2
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

import numpy as np
import tensorflow as tf
import random as python_random

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

def reset_seeds(SEED = 624):
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

tf.config.experimental.enable_op_determinism()

2023-10-28 16:32:33.299909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class LSTM_predictor:
    """A class for predicting time series using LSTM (Long Short-Term Memory) models."""
    def __init__(self, X, Y):
        """
        Initializes a new instance of the LSTM_predictor class.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.
        """
        self.X, self.Y = self.__clean_data__(X, Y)
        self.features_size = X.shape[1]
        self.obs_size = X.shape[0]
        
        
    def __clean_data__(self, X, Y):
        """
        Cleans the input feature matrix and target values by removing rows with NaN values.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.

        Returns:
            ndarray: The cleaned input feature matrix.
            ndarray: The cleaned target values.
        """
        Z = np.concatenate((X, Y), axis = 1)
        Z = Z[~np.isnan(Z).any(axis=1), :]
        return Z[:, :-1], Z[:, -1]
        
    def prepare_data(self, train_test_split = 0.8, timestep = 1, look_forward = 1, inplace = False, scaled = True):
        """
        Prepares the data for training and testing the LSTM model.

        Args:
            train_test_split (float, optional): The fraction of data to be used for training.
                Defaults to 0.8.
            timestep (int, optional): The number of time steps to consider for each sample.
                Defaults to 3.
            inplace (bool, optional): Indicates whether to store the prepared data in the class
                instance or return it. Defaults to False.
            scaled (bool, optional): Indicates whether to scale the data. Defaults to True.

        Returns:
            tuple or None: If inplace=True, returns None. Otherwise, returns a tuple containing
            the prepared training and testing data as (X_train, Y_train, X_test, Y_test).
        """
        split_idx = int(self.obs_size*0.8)
        train_set = self.X[:-4-look_forward,:], self.Y[timestep+look_forward:-4]
        test_set = self.X[-4-timestep+1:,:], self.Y[-4+look_forward:]
        print(test_set[0])
        
        # Scale factors
        mean_X = np.mean(train_set[0], axis = 0)
        std_X = np.std(train_set[0], axis = 0)
        self.__scale_X__ = (mean_X, std_X)
        
        mean_Y = np.mean(train_set[1], axis = 0)
        std_Y = np.std(train_set[1], axis = 0)
        self.__scale_Y__ = (mean_Y, std_Y)
        
        sampled = []

        for X, Y in [train_set, test_set]:
            X_sampled, Y_sampled = [], []
            #print(X.shape)
            #print(Y.shape)
            print(len(X))

            for i in range(len(X)-timestep-look_forward+1):
                print(i)

                X_sampled.append((X[i:i+timestep] - self.__scale_X__[0])/self.__scale_X__[1])
                Y_sampled.append((Y[i] - self.__scale_Y__[0])/self.__scale_Y__[1])
            
            sampled.append((np.array(X_sampled),np.array(Y_sampled)))
        
        if inplace:
            self.X_train, self.Y_train = sampled[0]
            self.X_test, self.Y_test = sampled[1]
            print("x_train after prepare_data has shape of {}".format(self.X_train.shape))
            print("x_test after prepare_data has shape of {}".format(self.X_test.shape))
            
            return None
        else:
            return sampled
        
    def init_model(self, optimizer='adam', loss='mean_squared_error', kernel_regularizer=None, num_lstm_units = 64, num_lstm_units_2 = 64,num_dense_units = 25, learning_rate=0.001):
        """
        Initializes the LSTM model with the specified optimizer and loss function.

        Args:
            optimizer (str, optional): The optimizer to use. Defaults to 'adam'.
            loss (str, optional): The loss function to use. Defaults to 'mean_squared_error'.
        """
        reset_seeds()
        model = Sequential()
        model.add(LSTM(num_lstm_units, return_sequences=True,\
                       input_shape= (self.X_train.shape[1], self.features_size),\
                      kernel_regularizer=kernel_regularizer, activation = 'relu')) #added activation = "relu"
        model.add(LSTM(num_lstm_units_2, return_sequences=False, kernel_regularizer=kernel_regularizer, activation = 'relu'))
        model.add(Dense(num_dense_units))
        model.add(Dense(1))
        
        
        #opt = Adam(learning_rate=learning_rate) #to remove and default to the code directly below
        
        #if optimizer == 'adam':
            #opt = Adam(learning_rate=learning_rate)
        #elif optimizer == 'SGD':
            #opt = SGD(learning_rate=learning_rate)
        #else:
            #raise ValueError(f"Unsupported optimizer: {optimizer}")
        
        model.compile(optimizer = optimizer, loss = loss) # changed optimizer = optimizer to optimizer = opt
        self.model = model
        
        return None
    
    def fit_model_train(self, batch_size=100, epochs=50, callbacks=None):
        """
        Trains the LSTM model.

        Args:
            batch_size (int, optional): The batch size for training. 
            epochs (int, optional): The number of epochs to train for. 
        """
        self.test_loss_list = []
        self.pred_per_epoch = []
        history_train = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, verbose=1, callbacks = callbacks)
        self.train_loss_list = history_train.history['loss']  # Store loss values in the list 
       
        
    def predict(self):
        """Makes predictions using the trained LSTM model."""
        self.Y_pred = self.model.predict(self.X_test)
        #mse = mean_squared_error(self.Y_test, self.Y_pred)
        #mse_list.append(mse)
        

        
    def key_metrics(self,epoch=None):
        Y_actual = self.Y
        Y_pred = self.Y_pred if epoch == None else self.pred_per_epoch[epoch-1]
        mse = mean_squared_error(self.Y_test, Y_pred)
        mae = mean_absolute_error(self.Y_test, Y_pred)
        exp_ratio = explained_variance_score(self.Y_test, Y_pred)
        
        return Y_actual[-1],(Y_pred[-1]*self.__scale_Y__[1] + self.__scale_Y__[0]), mse, mae, exp_ratio 
    
    def cross_validation(self, n_splits = 5, batch_size=100, epochs=10, callback=False):
        """
        Perform cross-validation for the LSTM model.

        Parameters:
        - n_splits: Number of cross-validation splits (int, default=5)
        - batch_size: Number of samples per gradient update (int, default=100)
        - epochs: Number of epochs to train the model (int, default=50)
        - callback: Flag indicating whether to use custom callback during training (bool, default=True)

        Returns:
        - None
        """
        reset_seeds()
        tscv = TimeSeriesSplit(n_splits=5)
        
        cv_train_scores = []
        cv_val_scores = []
        
        print('Initiate cross-validation')
        i = 1

        for train_index, test_index in tscv.split(self.X_train):
            print(f'Running cross validation round {i}')
            self.temp_val_scores = []

            #print(self.X_train[train_index].shape)
            #print(self.X_train[test_index].shape)
            
            cv_X_train, cv_X_test = self.X_train[train_index], self.X_train[test_index]
            cv_y_train, cv_y_test = self.Y_train[train_index], self.Y_train[test_index]
            
            #print(cv_X_test.shape)
    
            # Fit your model on the training data
            history_train = self.model.fit(cv_X_train, cv_y_train, batch_size=batch_size,\
                                           epochs=epochs,\
                                           verbose = 2)
            cv_train_scores.append(history_train.history['loss'])
            cv_val_scores.append(self.temp_val_scores)
            print('==============================================')
            i+=1
            
        self.cv_train_scores = np.array(cv_train_scores).mean(axis = 0)
        self.cv_val_scores = np.array(cv_val_scores).mean(axis = 0)



In [7]:
sheet_name = ['20220526','20220825','20221124','20230223','20230525']
symbols = pd.read_excel("target_stock_universe.xlsx", sheet_name = sheet_name[-4]) # change here to get the other sheet
symbols['Ticker_Symbol'] = symbols['Ticker'].str.split().str[0]+".L"
symbols = symbols[symbols['Ticker_Symbol']!='HSV.L']
symbols = symbols[symbols['Ticker_Symbol']!='HBR.L']
symbols = symbols[symbols['Ticker_Symbol']!='TW/.L']
symbols = symbols[symbols['Ticker_Symbol']!='MDC.L']
symbols = symbols[symbols['Ticker_Symbol']!='HL/.L']

symbols_short = symbols['Ticker_Symbol'] # change here to get the full list of stock
data = price_volume_data(symbols_short.to_list(), '2017-08-26', '2022-08-30')
data.get_technical_indicators()
technical_indicators = ['Close', 'Volume', 'SMA_5', 'SMA_14', 'ATR_5', 'ATR_14', 'ADX_5', 'ADX_14', 'RSI_5','RSI_14', 'MACD', 'MACD_signal', 'BBANDS_upp', 'BBANDS_mid', 'BBANDS_low']

In [8]:
def get_technicals(technical_indicators, symbols_short, data):
    results = {}
    Y = {}
    X = {}
    for x in range(len(symbols_short)):
        results[x] = data.pv[f'{symbols_short.iloc[x]}']
        Y[x] = np.array(results[x]['Close']).reshape(-1,1)
        X[x] = np.array(results[x][technical_indicators])
    return results, Y, X

In [9]:
def run_LSTM(X,Y):
    results = {}
    for x in range(len(symbols_short)):
        model1 = LSTM_predictor(X[x], Y[x])
        model1.prepare_data(train_test_split = 0.8, timestep = 3, look_forward = 3, inplace = True)
        model1.init_model(optimizer='adam', loss='mean_squared_error')
        model1.cross_validation()
        model1.fit_model_train(epochs=30,batch_size=32)
        #return model1
        #model1.fit_model_train(epochs=30,batch_size=32,callbacks=[CustomCallback(model1)])
        model1.predict()
        #return model1
        model1.key_metrics()
        #model1.plot_prediction()
        results[x] = model1.key_metrics()
    return results
    

In [10]:
symbols_short

0      BKG.L
1     BLND.L
2      ICP.L
4      BME.L
5      DPH.L
6     SMDS.L
7     HWDN.L
8     ABDN.L
9      HIK.L
10     CCL.L
11    CTEC.L
12    INVP.L
14    FRAS.L
15    WEIR.L
16    JMAT.L
18    DARK.L
20     IGG.L
21     BEZ.L
22    DPLM.L
23    BBOX.L
24     EMG.L
25     IMI.L
26     HSX.L
27    TATE.L
28    INCH.L
29     SXS.L
30     DRX.L
31     CCC.L
32     DLN.L
33     RSW.L
34     DLG.L
Name: Ticker_Symbol, dtype: object

In [11]:
results, Y, X = get_technicals(technical_indicators, symbols_short, data)

In [12]:
metrics = run_LSTM(X,Y)

[[ 3.78757251e+03  2.60010000e+05  3.87067734e+03  3.93218316e+03
   9.12901764e+01  9.91025576e+01  5.69703665e+01  2.13984571e+01
   2.93540651e+01  4.24794161e+01 -4.48154490e+00  2.51973515e+01
   4.00936674e+03  3.87067734e+03  3.73198795e+03]
 [ 3.74105176e+03  4.13946000e+05  3.82533862e+03  3.91812528e+03
   9.10589708e+01  9.84619569e+01  6.24564316e+01  2.28562045e+01
   2.46548141e+01  4.01154081e+01 -1.70549385e+01  1.67468935e+01
   3.95540332e+03  3.82533862e+03  3.69527393e+03]
 [ 3.72457544e+03  2.29712000e+05  3.78325967e+03  3.89641807e+03
   8.27328570e+01  9.49595601e+01  6.68452837e+01  2.42098271e+01
   2.30230874e+01  3.92816158e+01 -2.80258799e+01  7.79233881e+00
   3.87459190e+03  3.78325967e+03  3.69192743e+03]
 [ 3.67417798e+03  6.68644000e+05  3.75500796e+03  3.86932080e+03
   8.14993988e+01  9.36457034e+01  7.12395728e+01  2.58635505e+01
   1.83736568e+01  3.67644713e+01 -4.03222737e+01 -1.83058370e+00
   3.87265027e+03  3.75500796e+03  3.63736565e+03]
 [ 3

2023-10-28 16:32:48.353256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [204]
	 [[{{node Placeholder/_11}}]]
2023-10-28 16:32:48.354162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [204]
	 [[{{node Placeholder/_11}}]]


3/3 - 3s - loss: 0.7516 - 3s/epoch - 1s/step
Epoch 2/10
3/3 - 0s - loss: 0.6141 - 19ms/epoch - 6ms/step
Epoch 3/10
3/3 - 0s - loss: 0.4809 - 20ms/epoch - 7ms/step
Epoch 4/10
3/3 - 0s - loss: 0.3352 - 21ms/epoch - 7ms/step
Epoch 5/10
3/3 - 0s - loss: 0.1907 - 20ms/epoch - 7ms/step
Epoch 6/10
3/3 - 0s - loss: 0.1110 - 21ms/epoch - 7ms/step
Epoch 7/10
3/3 - 0s - loss: 0.1009 - 19ms/epoch - 6ms/step
Epoch 8/10
3/3 - 0s - loss: 0.0836 - 20ms/epoch - 7ms/step
Epoch 9/10
3/3 - 0s - loss: 0.0710 - 20ms/epoch - 7ms/step
Epoch 10/10
3/3 - 0s - loss: 0.0613 - 19ms/epoch - 6ms/step
Running cross validation round 2
Epoch 1/10
5/5 - 0s - loss: 0.1249 - 35ms/epoch - 7ms/step
Epoch 2/10
5/5 - 0s - loss: 0.0722 - 36ms/epoch - 7ms/step
Epoch 3/10
5/5 - 0s - loss: 0.0686 - 31ms/epoch - 6ms/step
Epoch 4/10
5/5 - 0s - loss: 0.0546 - 30ms/epoch - 6ms/step
Epoch 5/10


2023-10-28 16:32:52.205614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [407]
	 [[{{node Placeholder/_11}}]]
2023-10-28 16:32:52.205980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [407]
	 [[{{node Placeholder/_11}}]]


5/5 - 0s - loss: 0.0526 - 35ms/epoch - 7ms/step
Epoch 6/10
5/5 - 0s - loss: 0.0474 - 73ms/epoch - 15ms/step
Epoch 7/10
5/5 - 0s - loss: 0.0455 - 35ms/epoch - 7ms/step
Epoch 8/10
5/5 - 0s - loss: 0.0412 - 35ms/epoch - 7ms/step
Epoch 9/10
5/5 - 0s - loss: 0.0388 - 33ms/epoch - 7ms/step
Epoch 10/10
5/5 - 0s - loss: 0.0366 - 31ms/epoch - 6ms/step
Running cross validation round 3
Epoch 1/10
7/7 - 0s - loss: 0.3608 - 43ms/epoch - 6ms/step
Epoch 2/10
7/7 - 0s - loss: 0.2065 - 56ms/epoch - 8ms/step
Epoch 3/10
7/7 - 0s - loss: 0.1274 - 43ms/epoch - 6ms/step
Epoch 4/10


...

11/11 - 0s - loss: 0.1023 - 70ms/epoch - 6ms/step
Epoch 4/10
11/11 - 0s - loss: 0.0972 - 65ms/epoch - 6ms/step
Epoch 5/10
11/11 - 0s - loss: 0.0933 - 65ms/epoch - 6ms/step
Epoch 6/10
11/11 - 0s - loss: 0.0910 - 64ms/epoch - 6ms/step
Epoch 7/10
11/11 - 0s - loss: 0.0871 - 63ms/epoch - 6ms/step
Epoch 8/10
11/11 - 0s - loss: 0.0859 - 67ms/epoch - 6ms/step
Epoch 9/10
11/11 - 0s - loss: 0.0823 - 65ms/epoch - 6ms/step
Epoch 10/10
11/11 - 0s - loss: 0.0809 - 64ms/epoch - 6ms/step
Epoch 1/30
37/39 [===========================>..] - ETA: 0s - loss: 0.1492

2023-10-28 16:39:18.921919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1218]
	 [[{{node Placeholder/_11}}]]
2023-10-28 16:39:18.922468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1218]
	 [[{{node Placeholder/_11}}]]


39/39 [==============================] - 0s 4ms/step - loss: 0.1469
Epoch 2/30
39/39 [==============================] - 0s 5ms/step - loss: 0.1324
Epoch 3/30
39/39 [==============================] - 0s 5ms/step - loss: 0.1146
Epoch 4/30
39/39 [==============================] - 0s 4ms/step - loss: 0.1142
Epoch 5/30
39/39 [==============================] - 0s 4ms/step - loss: 0.1096
Epoch 6/30
39/39 [==============================] - 0s 4ms/step - loss: 0.1020
Epoch 7/30
39/39 [==============================] - 0s 4ms/step - loss: 0.1072
Epoch 8/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0949
Epoch 9/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0935
Epoch 10/30
39/39 [==============================] - 0s 5ms/step - loss: 0.0875
Epoch 11/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0928
Epoch 12/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0853
Epoch 13/30
39/39 [==============================] - 0s 4ms/

2023-10-28 16:39:24.270838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-28 16:39:24.271154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]


1/1 [==============================] - 0s 303ms/step


In [13]:
metrics_copy = pd.DataFrame(metrics)

In [14]:
metrics_copy.to_csv("prediction_result_3days_20220825.csv")

In [ ]:
###########################

In [ ]:
metrics_copy

In [ ]:
#market share NEAR rank date

def extract_sp(stocks, start, end):
    def data(ticker):
        return(yf.Ticker(ticker).get_shares_full(start= start, end=end))
    FAANG_Stock = map(data, stocks)
    return(pd.concat(FAANG_Stock,
                     keys = stocks,
                     names = ["Company", "Date"]
                    )
          )


tickers = symbols_short

result_ = extract_sp(tickers,start="2015-08-26", end="2022-08-30")
result_df = pd.DataFrame(result_, columns = ['Market Share'])
market_share_df =\
(
    result_df
    .reset_index()
    .sort_values(by='Date', ascending = False)
    .drop_duplicates(subset='Company', keep='first')

)

market_share_df

In [ ]:
market_share_df["Market Share"]

In [ ]:
shares_outstanding = pd.DataFrame(data.shares)

In [ ]:
shares_outstanding = shares_outstanding.T

In [ ]:
shares_outstanding.columns = symbols_short.to_list()

In [ ]:
shares_outstanding

In [ ]:
combined = pd.concat([metrics_copy,shares_outstanding], axis=0, join='outer')

In [ ]:
combined = combined.T

In [ ]:
combined

In [ ]:
combined['final_marketcap'] = combined[0]*combined['Actual Price']

In [ ]:
combined = combined.rename(columns={0: 'Market Share'})

In [ ]:
combined

In [ ]:
sorted_df = combined.sort_values(by='final_marketcap')

In [ ]:
sorted_df

In [ ]:
sorted_df.to_csv("results_stocks_comparison_20230523.csv")

In [ ]:
#################